In [1]:
import spacy
import spacy.cli
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
import gc

In [2]:
# load pickle to dataframe
filename = 'hansard-speeches-post2010.pkl'
with open(filename, 'rb') as f:
    df = pickle.load(f)

In [3]:
# spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm')#, disable=["parser"])

# Lemmatize

In [4]:
nlp.Defaults.stop_words.add("rose")

In [ ]:
def lemmatize(doc, stop_words=nlp.Defaults.stop_words):
    '''Takes a spacy doc and lemmatizes each of the tokens
    
    Takes alphanumeric tokens whose lowercase form does not appear in stop_words
    and returns their lowercase lemmatized form
    '''
    lemmas = [token.lemma_.lower() for token in doc if token.is_alpha and token.text.lower() not in stop_words]
    return lemmas

In [31]:
def process_chunk(texts, batch_size=50):
    '''returns list, each item of which is a list of lemmas'''
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=batch_size):
        preproc_pipe.append(lemmatize(doc))
    return preproc_pipe

def chunker(iterable, chunksize):
    '''Takes an iterable and returns a list of chunks of the iterable'''
    return (iterable[pos: pos + chunksize] for pos in range(0, len(iterable), chunksize))

def flatten(list_of_lists):
    '''Flatten a list of lists to a list'''
    return [item for sublist in list_of_lists for item in sublist]

def process_parallel(texts, chunksize=100, n_jobs=-1):
    executor = Parallel(n_jobs=n_jobs, backend='multiprocessing', prefer='processes')
    do = delayed(process_chunk)
    tasks = (do(chunk) for chunk in chunker(texts, chunksize=chunksize))
    result = executor(tasks)
    return flatten(result)

In [ ]:
# for training on GPU - no parallelization
# df['lemmas'] = process_chunk(df.speech, batch_size=500)

In [32]:
%%time
# for training on CPU - parallelization
df['lemmas'] = process_parallel(df.speech, chunksize=100, n_jobs=15)

MaybeEncodingError: Error sending result: '[[[I have to acquaint the House that this House has this day attended Her Majesty in the House of Peers, and that Her Majesty was pleased to make a Most Gracious Speech from the Throne to both Houses of Parliament, of which I have, for greater accuracy, obtained a copy., I shall direct that the terms of the Gracious Speech be printed in the Votes and Proceedings., Copies are available in the Vote Office., The Gracious Speech was as follows:My Lords and Members of the House of Commons My Government's legislative programme will be based upon the principles of freedom, fairness and responsibility., The first priority is to reduce the deficit and restore economic growth., Action will be taken to accelerate the reduction of the structural budget deficit., A new Office for Budget Responsibility will provide confidence in the management of the public finances., The tax and benefits system will be made fairer and simpler., Changes to National Insuranc

In [19]:
with open('hansard-speeches-post2010-lemmatized.pkl', 'wb') as f:
    pickle.dump(df, f)